In [2]:
import pulp as p
import numpy as np
from itertools import combinations

### สุ่ม Matrix แนวทแยงทั้งหมดที่เป็นไปได้ที่ประกอบไปด้วย -1 และ 1

In [3]:
def all_diag_combination(dim):
    possible_combination = set(combinations([-1, 1] * dim ,dim))
    return list(np.diag(item) for item in possible_combination)

### สร้าง Linear programming โดยเริ่มจาก lower bound ---> Upper bound

In [10]:
Cc = np.array([[-1], [-2], [3], [4]])                                                                           #จุดกึ่งกลางของแต่ละสมาชิกในเมทริกซ์
Ac = np.array([[5, 6, -7, 8],  
    [10, -11, 12, 13]])
Bc = np.array([[-9], [14]])
Sol_u = [ ]

for i in range(14,15):                                                                                                  #วน 25 รอบ
    
#################################### LOWER BOUND CASE ###########################################    
    
    lp_prob = p.LpProblem('Problem', p.LpMinimize)                                                   #กำหนดชื่อว่า ''Problem" และกำหนดว่าเป็น Minimization
    w = p.LpVariable("w", lowBound = 0)                                                                  #สร้างตัวแปร และ กำหนดให้แต่ละตัวแปรมีค่ามากกว่าหรือเท่ากับ 0
    x = p.LpVariable("x", lowBound = 0)                                                                     
    y = p.LpVariable("y", lowBound = 0)
    z = p.LpVariable("z", lowBound = 0)
    v = np.array([[w, x, y, z]])                                                                                 #เมทริกซ์ตัวแปร


    Al = Ac - (np.array([[0.01, 0.01, 0.01, 0.01], [0.01, 0.01, 0.01, 0.01]])*i)               #ขอบเขตล่างของเมทริซ์ A
    Au = Ac + (np.array([[0.01, 0.01, 0.01, 0.01], [0.01, 0.01, 0.01, 0.01]])*i)            #ขอบเขตบนของเมทริซ์ A
    stl = Al.dot(v.transpose())                                                                                 #ขอบเขตล่างของเมทริซ์ A คูณกับ เมทริกซ์ของตัวแปร
    stu = Au.dot(v.transpose())                                                                               #ขอบเขตบนของเมทริซ์ A คูณกับ เมทริกซ์ของตัวแปร
    Bl = Bc - (np.array([[0.01], [0.01]])*i)                                                               #ขอบเขตล่างของเมทริซ์ B
    Bu = Bc + (np.array([[0.01], [0.01]])*i)                                                             #ขอบเขตบนของเมทริซ์ B
    lp_prob += v.dot(Cc - (np.array([[0.01, 0.01, 0.01, 0.01], 
                                     [0.01, 0.01, 0.01, 0.01]])*i).transpose())[0][0]                  #Objective Function
    
    lp_prob += stl[0][0] <= Bu[0][0]                                                                     #ขอบเขตล่างของ A คูณกับตัวแปร น้อยกว่าหรือเท่ากับ ขอบเขตบนของ B
    lp_prob += stl[1][0] <= Bu[1][0]
    lp_prob += stu[0][0] >= Bl[0][0]                                                                     #ขอบเขตบนของ A คูณกับตัวแปร น้อยกว่าหรือเท่ากับ ขอบเขตล่างของ B
    lp_prob += stu[1][0] >= Bl[1][0]

    print(lp_prob)
    
    status = lp_prob.solve()                                                                                   #แก้ปัญหา Linear Program ของกรณี Lower bound
    #print(p.LpStatus[status])
    
    

###################################### UPPER BOUND CASE #########################################



#for k in range(25):
    for l in range(4):
        Lp_prob = p.LpProblem('Problem', p.LpMinimize)                                            #กำหนดชื่อว่า ''Problem" และกำหนดว่าเป็น Minimization
        a = p.LpVariable("a", lowBound = 0)                                                            #สร้างตัวแปร และ กำหนดให้แต่ละตัวแปรมีค่ามากกว่าหรือเท่ากับ 0
        b = p.LpVariable("b", lowBound = 0)
        c = p.LpVariable("c", lowBound = 0)
        d = p.LpVariable("d", lowBound = 0)
        v = np.array([[a, b, c, d]])
        Aye = Ac - all_diag_combination(2)[l].dot(np.array([[0.01, 0.01, 0.01, 0.01],   
                                                            [0.01, 0.01, 0.01, 0.01]])*i)                     #สร้างเมทริกซ์ Aye
        By = Bc + all_diag_combination(2)[l].dot(np.array([[0.01], [0.01]])*i)            #สร้างเมทริกซ์ By
    
        Lp_prob += v.dot(Cc + (np.array([[0.01, 0.01, 0.01, 0.01],
                                         [0.01, 0.01, 0.01, 0.01]])*i).transpose())[0][0]            #Objective Function
        Lp_prob += Aye[0].dot(v.transpose())[0] >= By[0][0]                                 #(Aye)(x) = By
        Lp_prob += Aye[0].dot(v.transpose())[0] <= By[0][0]
        Lp_prob += Aye[1].dot(v.transpose())[0] >= By[1][0]
        Lp_prob += Aye[1].dot(v.transpose())[0] <= By[1][0]

        #print(Lp_prob)
        status = Lp_prob.solve()                                                                           #แก้ปัญหา Linear Program ของกรณี Upper bound
        #Sol_u.append(p.value(Lp_prob.objective))                                                  #ใส่คำตอบที่คำนวณ 25 รอบลงในลิสต์ Sol_u
        #print('i =',i, ': %.4f' %(p.value(Lp_prob.objective)))
    #print('i =',i, ': %.4f' %(p.value(lp_prob.objective)))
    #print('i =',i, ': LB = %.4f' %(p.value(lp_prob.objective)) ,', UB = %.4f' %(max(Sol_u)))                                                              #แสดงคำตอบ
    #print(p.value(Lp_prob))
    print(p.LpStatus[status])

Problem:
MINIMIZE
-1.1400000000000001*w + -2.14*x + 2.86*y + 3.86*z + 0.0
SUBJECT TO
_C1: 4.86 w + 5.86 x - 7.14 y + 7.86 z <= -8.86

_C2: 9.86 w - 11.14 x + 11.86 y + 12.86 z <= 14.14

_C3: 5.14 w + 6.14 x - 6.86 y + 8.14 z >= -9.14

_C4: 10.14 w - 10.86 x + 12.14 y + 13.14 z >= 13.86

VARIABLES
w Continuous
x Continuous
y Continuous
z Continuous

Infeasible
